In [1]:
using POMDPs, POMDPTools, QuickPOMDPs, MCTS, DiscreteValueIteration, POMDPSimulators, POMDPModels, POMDPPolicies
using Distributions, Combinatorics, StaticArrays, D3Trees, Random
using FileIO, JLD2, TickTock

In [2]:
global Number_level = 4;
global Action_num = 4;

States=Array((1:Number_level))
Observations=Array((1:Number_level))

using POMDPs, QuickPOMDPs, POMDPModelTools, POMDPSimulators, QMDP

gamma=0.95

Observation_matrix=[
    0.8 0.2 0.0 0.0;
    0.1 0.8 0.1 0.0;
    0.0 0.2 0.8 0.0;
    0.0 0.0 0.0 1.0
]

Transition_matrix = [
    0.8571 0.1429 0.0    0.0;
    0.0    0.8571 0.1429 0.0;
    0.0    0.0    0.8    0.2;
    0.0    0.0    0.0    1.0
]
T=zeros(Number_level,Number_level,3);
T[:,:,1].=Transition_matrix;
T[:,:,2].=Transition_matrix;
for j in 1:Number_level
    T[j,:,3].=Transition_matrix[1,:];
    end
T[Number_level,:,1].=0.0
T[Number_level,Number_level,1]=1.0
T[Number_level,:,2].=Transition_matrix[1,:]

4-element view(::Array{Float64, 3}, 4, :, 2) with eltype Float64:
 0.8571
 0.1429
 0.0
 0.0

In [3]:
Observation_matrix[1,:]

4-element Vector{Float64}:
 0.8
 0.2
 0.0
 0.0

In [4]:
SparseCat(Observations, Observation_matrix[1,:])

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [5]:
# Observation_matrix=[
#     0.65  0.25 0.1   0.0;
#     0.175 0.65 0.175 0.0;
#     0.1   0.25 0.65  0.0;
#     0.0   0.0  0.0   1.0
# ]

In [6]:
T

4×4×3 Array{Float64, 3}:
[:, :, 1] =
 0.8571  0.1429  0.0     0.0
 0.0     0.8571  0.1429  0.0
 0.0     0.0     0.8     0.2
 0.0     0.0     0.0     1.0

[:, :, 2] =
 0.8571  0.1429  0.0     0.0
 0.0     0.8571  0.1429  0.0
 0.0     0.0     0.8     0.2
 0.8571  0.1429  0.0     0.0

[:, :, 3] =
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0
 0.8571  0.1429  0.0  0.0

In [7]:
NumberUnits=20
limit=12
failure_penalty = -1200.0;
maintenance_penalty = -100.0;
setup_cost = -300.0;
normal_operation = 0.0;
system_penalty=-2000

-2000

In [8]:
# In iterations, belief needs to be "manually" updated using updater(policy::Policy), 
# then call decision function using policy and belief state


In [9]:
SparseCat(Observations, Observation_matrix[1,:])

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [10]:
Initialstates = zeros(Number_level)
Initialstates[1]=1.0
Initialstates

4-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0

In [11]:
Initialstate = SparseCat(States, Initialstates)

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 1.0   
   2 ┤ 0.0                                      
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [12]:
# Initialobss = zeros(Number_level)
# Initialobss[1]=1.0
# Initialobss

In [13]:
Initialobs = SparseCat(Observations, Observation_matrix[1,:])

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [14]:
multiunit = QuickPOMDP(
    states=States,
    actions=Array((1:Action_num)),
    observations = Observations,
    #initialstate = repeat(1:1,NumberUnits),
    initialstate = Initialstate,
    #initialobs = repeat(1:1,NumberUnits),
    initialobs = Initialobs,
    discount = gamma,

    transition = function(s,a)
        if a == 1 # NO REPAIR, NO SYSTEM FAILURE
            #crd = Categorical(T[s,:,1]);
            #s = rand(crd);
            return SparseCat(States, T[s,:,1])
        end
        if a == 2 # UNIT NOT REPAIRED UNLESS FAILURE BUT REPAIR IN SYSTEM
            if s == Number_level
                s = 1
                #crd = Categorical(T[s,:,2]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,2])
            else
                #crd = Categorical(T[s,:,2]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,2])
            end
        end
        if a == 3 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM, NO SYSTEM FAILURE
            if s==Number_level
                s=1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,3])
            else
                s = 1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,3])
            end
        end
        if a == 4 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM,SYSTEM FAILED
            if s==Number_level
                s=1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,3])
            else
                s = 1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return SparseCat(States, T[s,:,3])
            end
        end
    end,

    observation = function(a,s)
        return SparseCat(Observations, Observation_matrix[s,:])
    end,
        
    reward = function (s, a)
        if a == 1 # NO REPAIR, NO SYSTEM FAILURE
            if s == Number_level
                return system_penalty/(limit)
            else
                return normal_operation
            end
            #crd = Categorical(T[s,:,1]);
            #s = rand(crd);
        end
        if a == 2 # UNIT NOT REPAIRED UNLESS FAILURE BUT REPAIR IN SYSTEM, NO SYSTEM FAILURE PENALTY
            if s == Number_level
                #s = 1
                #crd = Categorical(T[s,:,2]);
                #s = rand(crd);
                return failure_penalty +  setup_cost/NumberUnits + normal_operation
            else
                # Here it doesn't matter whether systems fails or not, since C_b does not apply to functional units
                return normal_operation +  setup_cost/NumberUnits
            end
        end
        if a == 3 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM, NO SYSTEM FAILURE PENALTY
            if s==Number_level
                #s=1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return failure_penalty +  setup_cost/NumberUnits + normal_operation
            else
                # Here it doesn't matter whether systems fails or not, since C_b does not apply to functional units
                return maintenance_penalty +  setup_cost/NumberUnits+ normal_operation 
            end
        end
        if a == 4 # UNIT REPAIRED AND MAINTENANCE IN SYSTEM,SYSTEM FAILED
            if s==Number_level
                #s=1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return failure_penalty +  setup_cost/NumberUnits + normal_operation + system_penalty/(limit)
            else
                #s = 1
                #crd = Categorical(T[s,:,3]);
                #s = rand(crd);
                return maintenance_penalty +  setup_cost/NumberUnits + normal_operation
            end
        end
    end,
    

)

QuickPOMDP{Base.UUID("f2e4cc47-3705-45a2-a25f-c9963dd4486e"), Int64, Int64, Int64, @NamedTuple{stateindex::Dict{Int64, Int64}, isterminal::Bool, obsindex::Dict{Int64, Int64}, states::Vector{Int64}, observations::Vector{Int64}, discount::Float64, initialobs::SparseCat{Vector{Int64}, Vector{Float64}}, actions::Vector{Int64}, observation::var"#2#5", actionindex::Dict{Int64, Int64}, initialstate::SparseCat{Vector{Int64}, Vector{Float64}}, transition::var"#1#4", reward::var"#3#6"}}((stateindex = Dict(4 => 4, 2 => 2, 3 => 3, 1 => 1), isterminal = false, obsindex = Dict(4 => 4, 2 => 2, 3 => 3, 1 => 1), states = [1, 2, 3, 4], observations = [1, 2, 3, 4], discount = 0.95, initialobs = SparseCat{Vector{Int64}, Vector{Float64}}([1, 2, 3, 4], [0.8, 0.2, 0.0, 0.0]), actions = [1, 2, 3, 4], observation = var"#2#5"(), actionindex = Dict(4 => 4, 2 => 2, 3 => 3, 1 => 1), initialstate = SparseCat{Vector{Int64}, Vector{Float64}}([1, 2, 3, 4], [1.0, 0.0, 0.0, 0.0]), transition = var"#1#4"(), reward = var"

In [15]:
initialstate(multiunit)

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 1.0   
   2 ┤ 0.0                                      
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [16]:
import Pkg; Pkg.add("NativeSARSOP")
using POMDPs
using NativeSARSOP
using POMDPModels



   Resolving package versions...
  No Changes to `C:\Users\hhu98\.julia\environments\v1.11\Project.toml`
  No Changes to `C:\Users\hhu98\.julia\environments\v1.11\Manifest.toml`


In [17]:
ordered_states(multiunit)

4-element Vector{Int64}:
 1
 2
 3
 4

In [18]:
length(states(multiunit))

4

In [19]:
#import Pkg; Pkg.add("IncrementalPruning")

In [20]:
# using IncrementalPruning
# solver = PruneSolver() # set the solver

# policy = solve(solver, multiunit)

In [21]:
#value(policy,[0.1,0.8,0.1,0.0])

In [22]:
solver = SARSOPSolver()
policy = solve(solver, multiunit)
# Calling observation

AlphaVectorPolicy{QuickPOMDP{Base.UUID("f2e4cc47-3705-45a2-a25f-c9963dd4486e"), Int64, Int64, Int64, @NamedTuple{stateindex::Dict{Int64, Int64}, isterminal::Bool, obsindex::Dict{Int64, Int64}, states::Vector{Int64}, observations::Vector{Int64}, discount::Float64, initialobs::SparseCat{Vector{Int64}, Vector{Float64}}, actions::Vector{Int64}, observation::var"#2#5", actionindex::Dict{Int64, Int64}, initialstate::SparseCat{Vector{Int64}, Vector{Float64}}, transition::var"#1#4", reward::var"#3#6"}}, Int64}(QuickPOMDP{Base.UUID("f2e4cc47-3705-45a2-a25f-c9963dd4486e"), Int64, Int64, Int64, @NamedTuple{stateindex::Dict{Int64, Int64}, isterminal::Bool, obsindex::Dict{Int64, Int64}, states::Vector{Int64}, observations::Vector{Int64}, discount::Float64, initialobs::SparseCat{Vector{Int64}, Vector{Float64}}, actions::Vector{Int64}, observation::var"#2#5", actionindex::Dict{Int64, Int64}, initialstate::SparseCat{Vector{Int64}, Vector{Float64}}, transition::var"#1#4", reward::var"#3#6"}}((stateinde

In [23]:
value(policy,[0.0,0.0,0.0,1.0])

-1430.316871825709

In [24]:
value(policy,[0.1,0.1,0.8,0.0])

-330.31687182570914

In [25]:
b=[[0.0, 0.2, 0.8, 0.0] for j in 1:NumberUnits]
belief=b[1]
value(policy,belief)

-330.31687182570914

In [26]:
#transition function: transition(pomdp, state, action)
#return the transition distribution form the current state-action pair
transition(multiunit, 3,1)

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤ 0.0                                      
   2 ┤ 0.0                                      
   3 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   4 ┤■■■■■■■■■ 0.2                             
     └                                        ┘ 

In [27]:
# observation function: observation(pomdp, action,statep)
# return the observation distribution
POMDPs.observation(multiunit,1,2)

               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■ 0.1                                  
   2 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   3 ┤■■■■ 0.1                                  
   4 ┤ 0.0                                      
     └                                        ┘ 

In [28]:
s=1
new_a = 1
p_observation = POMDPs.observation(multiunit,new_a,s)


               SparseCat distribution           
     ┌                                        ┐ 
   1 ┤■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■ 0.8   
   2 ┤■■■■■■■■■ 0.2                             
   3 ┤ 0.0                                      
   4 ┤ 0.0                                      
     └                                        ┘ 

In [29]:
# use pdf function to get prob of certain entry
new_obs=2
pdf(p_observation,new_obs)

0.2

In [30]:
function generative(s,a,rng=23) # calculate TRUE COMPONENT-LEVEL reward r_i(s_i,a_i) and get evolving states
    if a==1 # No maintenance in the system, no system failure
        crd=Categorical(T[s,:,1])
        s=rand(crd)
        r=normal_operation
        obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
        obs = rand(obs_distribution)
        return (sp=s,r=r,o=obs)
    end

    if a==2 # Maintenance occurs in the system, no system failure, corrective maintenance for the unit
        if s==Number_level
            crd=Categorical(T[1,:,2])
            s=rand(crd)
            r=failure_penalty  + normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            crd = Categorical(T[s,:,2])
            s = rand(crd)
            r = normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end

    if a==3 # Maintenance occurs in the system, no system failure, preventive maintenance for the unit
        if s==Number_level
            crd=Categorical(T[1,:,3])
            s=rand(crd)
            r=failure_penalty  + normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            crd = Categorical(T[s,:,3])
            s = rand(crd)
            r = maintenance_penalty+normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end

    if a==4 # maintenance in system, system failure occurs, preventive replacement for the unit
        if s==Number_level
            crd = Categorical(T[1,:,3])
            s = rand(crd)
            r=failure_penalty+ normal_operation
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        else
            r = maintenance_penalty+normal_operation
            crd = Categorical(T[1,:,3])
            s = rand(crd)
            obs_distribution = SparseCat(Observations, Observation_matrix[s,:])
            obs = rand(obs_distribution)
            return (sp=s,r=r,o=obs)
        end
    end
end

        
        
    

generative (generic function with 2 methods)

#### Belief updater

$$b'(s_j)=P(s_j\vert o,a,b)=\dfrac{P(o\vert s_j,a)\sum_{s_i\in \mathcal{S}}P(s_j\vert s_i,a)b(s_i)}{\sum_{s_j\in \mathcal{S}}P(o\vert s_j,a)\sum_{s_i\in \mathcal{S}}P(s_j\vert s_i,a)b(s_i)}$$

In [31]:
function update_belief(b,a,o,pomdp) 
    # NOTE: 
    # in input, b is belief from last epoch (before a)
    # a is the last action we took
    # o is the new observation after a
    updated_belief = zeros(length(b))
    if o==4
        updated_belief[length(states(pomdp))]=1.0
        return updated_belief
    end
    #if ((a==2)&&(o==4)) ||(a==3) ||(a==4)
    #if (a==2) ||(a==3) ||(a==4)
    if (a==3) ||(a==4)
    # We assume perfect maintenance and it is know to decision maker(belief updater)
        for sp in 1: length(states(pomdp))
            p_observation = POMDPs.observation(pomdp,a,sp) 
            p_obs = pdf(p_observation,o)
            # Transition probability distribution is T[1,:,1]
            p_trans = pdf(transition(pomdp,1,1),sp) 
            updated_belief[sp] = p_obs * p_trans
        end
        updated_belief ./= sum(updated_belief)
        return updated_belief
    end
    
    for sp in 1:length(states(pomdp)) 
        p_observation = POMDPs.observation(pomdp,a,sp) 
        p_obs = pdf(p_observation,o)
        p_trans = 0.0
        for s in 1:length(states(pomdp))
            #trans = transition(pomdp, s,a)
            #p_trans += pdf(transition(pomdp, s,a),sp) * pdf(b, s)
            # p_trans += T[s,sp,a] * pdf(b, s)
            p_trans += pdf(transition(pomdp,s,a),sp) * b[s]
        end
        updated_belief[sp] = p_obs * p_trans
    end
        # Normalize the updated belief state to ensure it sums up to 1
        updated_belief ./= sum(updated_belief)
    return updated_belief
end


update_belief (generic function with 1 method)

In [32]:
# s=1
# sp=2
# a=1
# pdf(transition(multiunit, s,a),sp)

In [33]:
# updated_belief_state = update_belief(SparseCat(States, [0.1,0.9,0.0,0.0]), 2, 4, multiunit)

In [34]:
# updated_belief_state = update_belief([0.0,0.1,0.9,0.0], 2, 2, multiunit)

In [35]:
# #updated_belief_state = update_belief(SparseCat(States, [0.0,0.1,0.9,0.0]), 2, 2, multiunit)
# updated_belief_state = update_belief([0.0,0.1,0.9,0.0], 2, 3, multiunit)
# # result of distinguishing a=2 taken under different states

In [36]:
# updated_belief_state = update_belief([0.1,0.9,0.0,0.0], 1, 1, multiunit)

### System-level decision making

In [37]:
function decision(policy, o, b)
# initialize system-level action vector
    Q=Qfnc(policy,b) # Q is an N*A matrix for each unit-action pair
    a_final = repeat(1:1,NumberUnits)
    failed_components = 0
    for i in 1:NumberUnits # numerate over units
        if o[i]==Number_level # failure state is self-announcing
            failed_components+=1
        end
    end
    if failed_components >= limit # System fails
        for i in 1:NumberUnits
            if o[i]==Number_level
                a_final[i]=4
            else
                a_final[i]=argmax(a->Q[i,a],[2,3]) # If the unit does not fail, the case is the same as a=2,3
            end
        end
        return a_final
    end

    # Based on belief, compare the expected total rewards of setting up maintenance or not
    D0=0.0
    a0=repeat(1:1,NumberUnits)
    for i in 1:NumberUnits
        D0+=Q[i,1]
    end

    D1=0.0
    a1=repeat(1:1,NumberUnits)
    for i in 1:NumberUnits
        a1[i]=argmax(a->Q[i,a],[2,3])
        if o[i]==Number_level
            a1[i]=3
        end
        D1+=Q[i,a1[i]]
    end

    if !(3 in a1)
        return a0
    end

    if D0 > D1
        return a0
    else
        return a1
    end
end


    
    
    


decision (generic function with 1 method)

# $Q$-function
After obtaining $V^*(b)$ from policy, we calculate optimal $Q$-function by
$$Q^*(b,a)=R(b,a)+\gamma  \sum_{o'\in \mathcal{O}} \eta(o'|b,a)V^*(b')$$
where expected immediate reward of taking action $a$ in belief state $b$ is $$R(b,a)=\sum_{s\in \mathcal{S}}R(s,a) b(s),$$
and prior probability of observing $o'$ after taking action $a$ is $$\eta(o'|b,a)=\sum_{s'\in\mathcal{S}} P(o'|s',a)\sum_{s\in\mathcal{S}}T(s'|s,a)b(s)$$

In [38]:
function Qfnc(policy,b)
    epsilon = 1e-4
    Q=zeros((NumberUnits,4)) 
    # NOTE: Q is defined for each unit (according to their beliefs), not state
    for i in 1:NumberUnits
        belief=b[i]
        # if i == 1
        #     println("The current belief is ",belief)
        # end
        for a in 1:4
            # if i==1
            # println("Take action ",a)
            # end
            Q_value=0
            for s in 1:Number_level
                Q_value+=belief[s] * POMDPs.reward(multiunit,s,a)
            end
            for o in 1:Number_level
                # if i==1
                # println("prior probability of observing ",o," after taking action ",a)
                # end
            # the o here is the observation we expect to obtain at next time step, 
            # i.e., after taking the action we are deciding
            # so we don't need input current o
                o_prior = 0 
                for sp in 1:Number_level
                    p_sp=0
                    for s in 1:Number_level
                        p_sp+=pdf(transition(multiunit,s,a),sp) * belief[s]
                        #p_sp+=T[s,sp,a] * belief[s]
                    end
                    p_sp=p_sp * Observation_matrix[sp,o]
                    o_prior+=p_sp
                end
                #println(o_prior)
                bp=update_belief(belief,a,o,multiunit)
                # if i == 1
                #     println(o_prior)
                #     println("next time step belief vector after observing ",o)
                #     println(bp)
                # end
                if o_prior > epsilon
                    Q_value += gamma * o_prior * value(policy,bp)
                end
            end
            Q[i,a]=Q_value
        end
    end
    return Q
end
        
                
                

Qfnc (generic function with 1 method)

In [39]:
b=[[0.0, 0.2, 0.8, 0.0] for j in 1:NumberUnits]


20-element Vector{Vector{Float64}}:
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]
 [0.0, 0.2, 0.8, 0.0]

In [40]:
Q=Qfnc(policy,b)

20×4 Matrix{Float64}:
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317
 -480.37  -495.37  -330.317  -330.317

In [41]:
b=[[0.2, 0.3, 0.5, 0.0] for j in 1:NumberUnits]

20-element Vector{Vector{Float64}}:
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]
 [0.2, 0.3, 0.5, 0.0]

In [42]:
o=rand(1:4,20)
for i in 1:10
    o[i]=4
end

In [43]:
decision(policy, o, b)

20-element Vector{Int64}:
 4
 4
 4
 4
 4
 4
 4
 4
 4
 4
 4
 3
 4
 4
 3
 4
 3
 3
 4
 3

In [44]:
# Define the fixed K values for each unit count
function get_fixed_K(units)
    if units == 60
        return 18
    elseif units == 50
        return 18
    elseif units == 40
        return 18
    elseif units == 30
        return 14
    elseif units == 25
        return 10  # Assuming a reasonable value for 25
    elseif units == 20
        return 10   # Assuming a reasonable value for 20
    else
        return convert(Int64, floor((units+1)/4))  # Fallback for any other values
    end
end

get_fixed_K (generic function with 1 method)

### Iteration

In [45]:
using DataFrames
# df= DataFrame(u=[],K=[],n=[],s=[],m=[],f=[],p=[],lamb=[],mean=[],std=[])
# Define the observation matrices with varying uncertainty levels
observation_matrices = [
    # Low Uncertainty
    [
        0.9  0.1  0.0  0.0;
        0.05 0.9  0.05 0.0;
        0.0  0.1  0.9  0.0;
        0.0  0.0  0.0  1.0
    ],
    # Original Uncertainty
    [
        0.8  0.2  0.0  0.0;
        0.1  0.8  0.1  0.0;
        0.0  0.2  0.8  0.0;
        0.0  0.0  0.0  1.0
    ],
    # High Uncertainty
    [
        0.7  0.3  0.0  0.0;
        0.15 0.7  0.15 0.0;
        0.0  0.3  0.7  0.0;
        0.0  0.0  0.0  1.0
    ],
    # Very High Uncertainty
    [
        0.6  0.4  0.0  0.0;
        0.2  0.6  0.2  0.0;
        0.0  0.4  0.6  0.0;
        0.0  0.0  0.0  1.0
    ],
    # Extreme Uncertainty
    [
        0.5  0.5  0.0  0.0;
        0.25 0.5  0.25 0.0;
        0.0  0.5  0.5  0.0;
        0.0  0.0  0.0  1.0
    ]
]

# Labels for the observation matrices (useful for output and plotting)
uncertainty_labels = ["Low", "Original", "High", "Very High", "Extreme"]

# Modified DataFrame with an additional column for uncertainty level
df = DataFrame(uncertainty=[], u=[], K=[], n=[], s=[], m=[], f=[], p=[], lamb=[], mean=[], std=[])


cost=[ 
    [0  -2400   -100    -1200 -2000]

] 



for units in [20, 25, 30, 40, 50, 60]

    # for K in 2:2:convert(Int64, floor((units+1)/2)) 
    global limit = get_fixed_K(units)
    global K = limit
          
    global NumberUnits=convert(Int64,units)
    # global limit=convert(Int64,K)
    #for lambda in [1,0.9,0.75,0.5,0.1,0.075,0.05,0.01,0.0075,0.005,0.0001]
    for lambda in 1:1:1
    global lamb=lambda
    for c in cost
        for obs_idx in 1:length(observation_matrices)
            global Observation_matrix = observation_matrices[obs_idx]
            global uncertainty_level = uncertainty_labels[obs_idx]
        tick()
        println(units)  
        print("K : ")
        println(K)
        global normal_operation,setup_cost,maintenance_penalty,failure_penalty,system_penalty=c
        println(c)  
        global NumberUnits=units
                println("Uncertainty Level: ")
                print(uncertainty_level)
# POMDP is set up
# CONTINUE to solve it and compute expected total rewards
policy = solve(solver, multiunit)
# Obtain Q function from Value function (given by policy)

trials=10000;
simsteps = 100;
results=zeros(trials)*0.1
Threads.@threads  for h in 1:trials
# Suppose we start from brand new states and we know it
s=repeat(1:1,NumberUnits)
# o=s 
o = rand(Initialobs, NumberUnits)
# belief = [[1.0, 0.0, 0.0, 0.0] for j in 1:NumberUnits]
belief = [Observation_matrix[1,:] for j in 1:NumberUnits]
R_sequence=[] 
r=repeat(1:1,NumberUnits)*1.0

# iteration
for k in 1:simsteps
    a=decision(policy,o,belief) # system-level decision making: action vector
    
    # SANITY CHECK
    # Update belief for each component individually
    #b=update_belief(b, a, o, multiunit) # what is the sequence of observing, updating belief, taking action?
    
    R_instant = 0.0
    global failed_components=0
    # When generating new state and new reward, we use true states
    for i in 1:NumberUnits # numerate over units
        if o[i]==Number_level # failure state is self-announcing
            failed_components+=1
        end
    s[i],r[i],o[i]=generative(s[i],a[i],234) #call generative function for underlying states
    # it might be better to use generative function to give o as well
    #o[i]=emission(s[i]) # observation generated by new underlying state
    belief[i]=update_belief(belief[i], a[i], o[i], multiunit)
    # sum up single-component cost
    R_instant += r[i]
    end
    # add setup cost
    if (2 in a) ||(3 in a) ||(4 in a)
        R_instant += setup_cost
    end
    # add system failure penalty
    if failed_components >= limit
        R_instant += system_penalty
    end
    # record instant reward of current time step
    append!(R_sequence, R_instant)
end
R_discounted=0
for k in 1:simsteps
    R_discounted=gamma * R_discounted+R_sequence[simsteps-k+1]
end
results[h]=R_discounted
end # end the 10000 trials

println(units)
            print("k : ")
            println(limit)
println(lambda)

println(mean(results))

println(std(results))
    pushfirst!(df, [
        uncertainty_level,  # Add the uncertainty level as the first column
        units, 
        limit, 
        normal_operation, 
        setup_cost, 
        maintenance_penalty, 
        failure_penalty, 
        system_penalty, 
        lamb, 
        mean(results), 
        std(results)
    ])
        
tock()

                    
                    
    end
    end        
end
end


20
K : 10


[ Info:  started timer at: 2025-04-11T15:00:48.525


[0 -2400 -100 -1200 -2000]
Uncertainty Level: 
Low20
k : 10
1
-12375.720751856577
1636.6470453347647
20
K : 10
[0 -2400 -100 -1200 -2000]
Uncertainty Level: 
Original

[ Info:          490.5995881s: 8 minutes, 10 seconds, 599 milliseconds
[ Info:  started timer at: 2025-04-11T15:08:59.276


20
k : 10
1
-12442.425575383284
1657.7358252885992
20
K : 10
[0 -2400 -100 -1200 -2000]
Uncertainty Level: 
High

...

[ Info:          955.3548872s: 15 minutes, 55 seconds, 354 milliseconds
[ Info:  started timer at: 2025-04-11T20:16:14.964


60
k : 18
1
-29260.78412334087
2418.5831391653364
60
K : 18
[0 -2400 -100 -1200 -2000]
Uncertainty Level: 
Extreme

[ Info:          994.5686602s: 16 minutes, 34 seconds, 568 milliseconds
[ Info:  started timer at: 2025-04-11T20:32:49.547


60
k : 18
1
-29807.88253768276
2433.823087405922


[ Info:         1012.2835098s: 16 minutes, 52 seconds, 283 milliseconds


In [46]:
df

Row,uncertainty,u,K,n,s,m,f,p,lamb,mean,std
,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,Extreme,60,18,0,-2400,-100,-1200,-2000,1,-29807.9,2433.82
2,Very High,60,18,0,-2400,-100,-1200,-2000,1,-29260.8,2418.58
3,High,60,18,0,-2400,-100,-1200,-2000,1,-28649.0,2365.99
4,Original,60,18,0,-2400,-100,-1200,-2000,1,-28955.7,2402.64
5,Low,60,18,0,-2400,-100,-1200,-2000,1,-30673.4,2443.53
6,Extreme,50,18,0,-2400,-100,-1200,-2000,1,-25836.5,2270.55
7,Very High,50,18,0,-2400,-100,-1200,-2000,1,-25371.1,2238.6
8,High,50,18,0,-2400,-100,-1200,-2000,1,-24899.5,2182.35
9,Original,50,18,0,-2400,-100,-1200,-2000,1,-24830.3,2209.06


In [47]:
using CSV
CSV.write("CW-POMDP.csv", df)

"CW-POMDP.csv"